In [ ]:
import numpy as np
from polartoolkit import fetch, utils
import verde as vd
import numpy as np
import os
os.environ["POLARTOOLKIT_HEMISPHERE"] = "south"

# Use source ID from Bedmachine to identify coordinates of known bed topography
# get bedmachine data id grid
number_to_source = {
    0: "no_data",
    1: "rema",
    2: "radar",
    7: "seismic",
    10: "multibeam",
}
dataid = fetch.bedmachine(layer="dataid").compute()

# drop grid cells of "no_data"
dataid_ds = dataid.where(dataid != 0, drop=True)

# turn into dataframe
dataid_df = (
    vd.grid_to_table(dataid_ds)
    .dropna()
    .rename(columns={"x": "easting", "y": "northing"})
)
dataid_df["source"] = dataid_df.dataid.map(number_to_source)
dataid_df = dataid_df[["easting", "northing", "dataid", "source"]]

# only keep points in ice shelf region
onshore_points= utils.points_inside_region(dataid_df, region)
onshore_points

In [ ]:
# get bedmap3 bed topography grid
bed_topography = fetch.bedmap3(layer="bed", region = region, reference="ellipsoid")

# show map of the points
fig = maps.plot_grd(bed_topography, coast=True)
fig.add_points(onshore_points, style="p.2p", fill="red")
fig.show()

# Get offshore points of known bathymetry

In [ ]:
import pandas as pd
import geopandas as gpd
import shapely

def polygons_to_points(
    polygons: gpd.GeoDataFrame,
    spacing: float = 100,
) -> pd.DataFrame:
    """
    Convert a geodataframe of polygons to a grid of points with a specified spacing.

    Parameters
    ----------
    polygons : geopandas.GeoDataFrame
        Geodataframe of polygons to convert to points
    spacing : float, optional
        Spacing between points, by default 100

    Returns
    -------
    pd.DataFrame
        Dataframe of points with columns "easting", "northing", and "geometry"
    """
    points_list = []
    for _, row in polygons.iterrows():
        points_list.append(polygon_to_points(row, spacing=spacing))

    return pd.concat(points_list)


def polygon_to_points(
    polygon: gpd.GeoSeries,
    spacing: float = 100,
) -> pd.DataFrame:
    """
    Convert a polygon shapefile to a grid of points with a specified spacing. Also
    include the vertices of the polygon.

    Parameters
    ----------
    polygon : gpd.GeoSeries
        Polygon shapefile to convert to points
    spacing : float, optional
        Spacing between points, by default 100

    Returns
    -------
    pd.DataFrame
        Dataframe of points with columns "easting", "northing", and "geometry"
    """

    # get bounds of polygon
    bounds = polygon.geometry.bounds

    # create grid of points
    x_coords = np.arange(bounds[0], bounds[2], spacing)
    y_coords = np.arange(bounds[1], bounds[3], spacing)
    points = [shapely.geometry.Point(x, y) for x in x_coords for y in y_coords]

    # Filter points within the polygon
    points_in_polygon = [point for point in points if polygon.geometry.contains(point)]

    # add vertices of polygon
    points_in_polygon.extend(
        [shapely.geometry.Point(x, y) for (x, y) in polygon.geometry.exterior.coords]
    )

    # add points along edges
    points_in_polygon.extend(
        [
            shapely.geometry.Point(x, y)
            for (x, y) in polygon.geometry.segmentize(spacing).exterior.coords
        ]
    )

    # convert to geodataframe
    points_gdf = gpd.GeoDataFrame(geometry=points_in_polygon)

    # add easting and northing columns
    points_gdf["easting"] = [p.x for p in points_gdf.geometry]
    points_gdf["northing"] = [p.y for p in points_gdf.geometry]

    return points_gdf

In [ ]:
# fetch IBCSO points for region
ibcso_points_gdf, ibcso_polygons_gdf = fetch.ibcso_coverage(region=region)

# drop points with TID of 45, meaning it's from gravity inversion
ibcso_points_gdf = ibcso_points_gdf[ibcso_points_gdf.dataset_tid != 45]

# convert polygons (mostly swath multibeam) to grid of points which are within the
# polygon with a grid spacing specified
polygon_points = None
if len(ibcso_polygons_gdf) > 0:
    try:
        polygon_points = polygons_to_points(
            ibcso_polygons_gdf,
            spacing=1e3,
        )
    except ValueError as e:
        print(e)
        print("Failed to convert polygons to points")

# combine IBCSO points and polygon points
offshore_points= pd.concat([ibcso_points_gdf, polygon_points])

# drop some columns
offshore_points= offshore_points.drop(columns=["geometry", "weight", "dataset_tid"])

offshore_points

In [ ]:
fig = maps.plot_grd(bed_topography, coast=True)
fig.add_points(offshore_points, style="p.2p", fill="red")
fig.show()

# Combine onshore and offshore points

In [ ]:
constraint_points =  pd.concat(
    [offshore_points, onshore_points],
    ignore_index=True,
    sort=False,
)

# drop unnecessary columns
constraint_points = constraint_points.drop(columns=["dataset_name", "dataid", "source"])
constraint_points

In [ ]:
fig = maps.plot_grd(bed_topography, coast=True)
fig.add_points(constraint_points, style="p.2p", fill="red")
fig.show()

In [ ]:
# Sample bed topography grid at these points

In [ ]:
# sample bedrock topography at points
constraint_points = profiles.sample_grids(constraint_points, bed_topography, sampled_name="bed_elevation")

# drop points without bed elevation
constraint_points = constraint_points.dropna(subset=["bed_elevation"])
constraint_points

In [ ]:
# Save to CSV

In [ ]:
constraint_points.to_csv("constraint_points.csv", index=False)